In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

plt.style.use('seaborn-notebook')

%matplotlib inline

In [2]:
orig_data = pd.read_csv('../data/kc_house_data.csv', index_col=0).dropna()
df = orig_data.copy()

In [3]:
   
    # Cleaning
    highquant_living = df['sqft_living'].quantile(0.99)
    highquant_lot = df['sqft_lot'].quantile(0.99)
    highquant_basement = df['sqft_basement'].quantile(0.99)
    highquant_patio = df['sqft_patio'].quantile(0.99)
    highquant_garage = df['sqft_garage'].quantile(0.99)
    
    df = df[df['sqft_living'] < highquant_living]
    df = df[df['sqft_living'] > 100] # Gets rid of abnormally low sqfts
    
    df = df[df['sqft_lot'] < highquant_lot]
    
    df = df[df['sqft_basement'] < highquant_basement]
    
    df = df[df['sqft_patio'] < highquant_patio]
    
    df = df[df['sqft_garage'] < highquant_garage]
    
    # Feature engineering
    df['age'] = 2021 - df['yr_built']

    df['bath_to_bed'] = df['bathrooms'] / df['bedrooms']
    df = df.replace([np.inf, -np.inf], df['bath_to_bed'].median()).fillna(df['bath_to_bed'].median()) # Replaces inf/-inf/nan values with median

    df['zip'] = df.address.apply(lambda x: x[x.rfind(',') - 5:x.rfind(',')])
    
    df['condition'] = df['condition'].replace({'Poor': 1, 
                                               'Fair': 2, 
                                               'Average': 3, 
                                               'Good': 4, 
                                               'Very Good': 5})
    df['grade'] = df['grade'].replace({'1 Cabin': 1, 
                                       '2 Substandard': 2, 
                                       '3 Poor': 3, 
                                       '4 Low': 4, 
                                       '5 Fair': 5,
                                       '6 Low Average': 6,
                                       '7 Average': 7,
                                       '8 Good': 8,
                                       '9 Better': 9,
                                       '10 Very Good': 10,
                                       '11 Excellent': 11,
                                       '12 Luxury': 12,
                                       '13 Mansion': 13,
                                      })
    


In [9]:
df['zip'].nunique()

396

In [4]:
renovated = historic[(historic["Renovation"] == 1)]
renovated_mean = renovated.price.mean()
no_renovation = historic[(historic["Renovation"] == 0)]
no_renovation_mean = no_renovation.price.mean()

label_renovation = ["Renovated", "No Renovation"]
values_renovation = [renovated_mean, no_renovation_mean]

fig, ax = plt.subplots( figsize=(12,6))
sns.barplot(ax=ax, x=label_renovation, y=values_renovation)

NameError: name 'historic' is not defined

In [ ]:
historic_corr = historic.corr()['price'].map(abs).sort_values(ascending=False)
historic_corr

In [ ]:
historic_pred1 = historic[['grade_num']]
target = historic['price']
historic_with_con = sm.add_constant(historic_pred1)

In [ ]:
model1 = sm.OLS(endog=target, exog=historic_pred1).fit()
model1.rsquared
#Grade accounts for 68% of variation in price

In [ ]:
model1.summary()